# Aries Basic Controller Example - Alice
# DID Exchange - Inviter

In this notebook we'll be initiating the aries [DID Exchange](https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange) protocol using the aries_basic_controller package. 

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's [notebook](http://localhost:8889)

<b>Carry on in Bob's notebook</b>
    
12. Accept Request for Connection
13. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [4]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8023
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)


### 3. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using PyPubSub. The default listeners are used to update state and print logs.

In [5]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([], defaults=True)

### 4. Use the controller to create an invitation from our agent


In [6]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f01680ae2d0>


ClientConnectorError: Cannot connect to host alice-agent:8021 ssl:default [Connect call failed ('172.25.0.5', 8021)]

Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit

### 5. Copy the invitation output from 4 and move over to the Bob notebook (localhost:8889) 

### 11. Accept Request for Connection

In [5]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

True
ACCEPT REQUEST
{'their_did': 'AJxZTGz5B4t8SLDK6vGZ8Y', 'created_at': '2020-07-09 10:20:37.368197Z', 'connection_id': '9f57c0ed-8aa9-4cbe-ab45-b520bef74ccd', 'initiator': 'self', 'invitation_mode': 'once', 'their_label': 'Bob', 'invitation_key': '5QbLLjSHmBJzM3K5cAkD9dNymGXuhYJ98Y5tU4fcofpG', 'state': 'response', 'accept': 'manual', 'updated_at': '2020-07-09 10:21:25.157030Z', 'my_did': 'Jbkw8xBk9JCn9wwoHAKPjp', 'routing_state': 'none'}
state response


### 12. Send Trust Ping to activate the conneciton

In [6]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '806a1294-0330-4578-98a0-71049381e99f'}


####  12.1 Check if connection is active

In [7]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

{'their_did': 'AJxZTGz5B4t8SLDK6vGZ8Y', 'created_at': '2020-07-09 10:20:37.368197Z', 'connection_id': '9f57c0ed-8aa9-4cbe-ab45-b520bef74ccd', 'initiator': 'self', 'invitation_mode': 'once', 'their_label': 'Bob', 'invitation_key': '5QbLLjSHmBJzM3K5cAkD9dNymGXuhYJ98Y5tU4fcofpG', 'state': 'active', 'accept': 'manual', 'updated_at': '2020-07-09 10:21:30.517540Z', 'my_did': 'Jbkw8xBk9JCn9wwoHAKPjp', 'routing_state': 'none'}
Is Active? active


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [8]:
response = await agent_controller.terminate()
print(response)

None
